# Installing dependencies

In [ ]:
!pip install -q -U transformers accelerate faker bitsandbytes

# Uploading model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from faker import Faker
import json
import re
from datetime import datetime, timedelta

MODEL_ID = "OpenLLM-Ro/RoLlama3-8b-Instruct-DPO-2025-04-23"

print(f"Se încarcă modelul {MODEL_ID} în mod 4-bit")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto", # Asta merge perfect pe GPU NVIDIA
        trust_remote_code=True
    )
    print("Model încărcat cu succes pe GPU!")
except Exception as e:
    print(f"Eroare încărcare: {e}")



# Template procura

In [ ]:
def get_template_procura(meta):
  text = f"""PROCURĂ SPECIALĂ

Subsemnatul {meta['mandant']['nume']}, cetățean român, domiciliat în {meta['mandant']['adresa']}, identificat cu C.I {meta['c.i.']}, C.N.P. {meta['mandant']['cnp']},
împuternicesc prin prezenta pe {meta['mandatar']['nume']}, cetățean român, domiciliat în {meta['mandatar']['adresa']}, C.N.P. {meta['mandatar']['cnp']},

Ca în numele meu şi pentru mine să se prezinte şi să mă reprezinte, cu puteri depline în vederea îndeplinirii tuturor formalităţilor necesare:
{meta['scop']}.

Eu, {meta['mandant']['nume']}, declar că am citit personal întregul cuprins al acestuia, i-am înţeles conţinutul şi consecinţele juridice, solicitând autentificarea prezentului înscris.

Mandatul este netransmisibil şi valabil până la data de {meta['termen']}.

Tehnoredactată la BIROU NOTARIAL PUBLIC, într-un exemplar original și 3 duplicate.

Mandant,
{meta['mandant']['nume']}
"""
  return text

# Template metadata

In [ ]:
fake = Faker('ro_RO')
def genereaza_metadata():
    scopuri = [
        "vânzarea apartamentului nr. 10, bloc A3, din București",
        "reprezentarea la succesiune și partaj voluntar",
        "înmatricularea autoturismului și reprezentarea la RAR",
        "ridicarea pensiei și administrarea conturilor bancare"
    ]
    return {
        "mandant": {"nume": fake.name(), "cnp": fake.ssn(), "adresa": fake.address().replace("\n", ", ")},
        "mandatar": {"nume": fake.name(), "cnp": fake.ssn(), "adresa": fake.address().replace("\n", ", ")},
        "scop": random.choice(scopuri),
        "termen": (datetime.now() + timedelta(days=365)).strftime("%d.%m.%Y"),
        "c.i.": fake.ssn()
    }

In [ ]:
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker('ro_RO')

def genereaza_date_complete():
    # Definim o listă extinsă de scenarii realiste din practica notarială
    scenarii = [
        # --- SCENARIUL 1: CĂLĂTORIE MINOR (DECLARAȚIE) ---
        {
            "tip_act": "DECLARAȚIE ACORD CĂLĂTORIE",
            "scop": "ieșire din țară minor neînsoțit de ambii părinți",
            "extra": {
                "minor": {"nume": fake.name(), "data_nasterii": "20.05.2016"},
                "insotitor": {"nume": fake.name(), "cnp": fake.ssn()}, # Poate fi bunic, mătușă etc.
                "destinatie": random.choice(["Grecia", "Turcia", "Spania", "Italia", "Marea Britanie"]),
                "perioada": "01.08.2025 - 20.08.2025",
                "motiv": "turism și vacanță"
            }
        },

        # --- SCENARIUL 2: AUTO (ÎNMATRICULARE) ---
        {
            "tip_act": "PROCURĂ SPECIALĂ AUTO",
            "scop": "reprezentare RAR și DRPCIV pentru înmatriculare",
            "extra": {
                "auto": {
                    "marca": random.choice(["Dacia Duster", "Ford Focus", "Volkswagen Golf", "Toyota Corolla"]),
                    "serie_sasiu": "".join(random.choices("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", k=17)),
                    "an_fabricatie": str(random.randint(2010, 2024))
                }
            }
        },

        # --- SCENARIUL 3: IMOBILIARE (VÂNZARE) ---
        {
            "tip_act": "PROCURĂ SPECIALĂ IMOBILIARĂ",
            "scop": "vânzare apartament",
            "extra": {
                "imobil": {
                    "tip": "apartament 2 camere",
                    "adresa": fake.address().replace("\n", ", "),
                    "nr_carte_funciara": str(random.randint(10000, 99999)),
                    "pret_minim": f"{random.randint(60, 150)}000 EURO"
                }
            }
        },

        # --- SCENARIUL 4: IMOBILIARE (CUMPĂRARE) ---
        {
            "tip_act": "PROCURĂ DE CUMPĂRARE",
            "scop": "achiziție imobil și semnare contract credit ipotecar",
            "extra": {
                "buget_maxim": f"{random.randint(80, 200)}000 EURO",
                "zona_preferata": "București sau Ilfov",
                "banca": random.choice(["Banca Transilvania", "BCR", "BRD", "ING Bank"])
            }
        },

        # --- SCENARIUL 5: SUCCESIUNE (MOȘTENIRE) ---
        {
            "tip_act": "PROCURĂ SUCCESORALĂ",
            "scop": "reprezentare la dezbaterea succesiunii",
            "extra": {
                "defunct": {"nume": fake.name(), "data_deces": (datetime.now() - timedelta(days=random.randint(10, 100))).strftime("%d.%m.%Y")},
                "notar_competent": "oricare notar public din raza Municipiului București",
                "optiune": "acceptarea pură și simplă a succesiunii"
            }
        },

        # --- SCENARIUL 6: CORPORATE (FIRMĂ) ---
        {
            "tip_act": "PROCURĂ SPECIALĂ ONRC",
            "scop": "înființare societate comercială (SRL)",
            "extra": {
                "nume_firma": f"{fake.word().upper()} SOLUTIONS SRL",
                "capital_social": "200 RON",
                "sediu_social": fake.address().replace("\n", ", ")
            }
        },

        # --- SCENARIUL 7: BANCAR (CONTURI) ---
        {
            "tip_act": "PROCURĂ BANCARĂ",
            "scop": "administrare conturi și ridicare carduri",
            "extra": {
                "banca": random.choice(["Raiffeisen Bank", "UniCredit", "CEC Bank"]),
                "operatiuni_permise": "depuneri, retrageri numerar, lichidare depozite, ridicare extrase de cont"
            }
        },

        # --- SCENARIUL 8: PENSIE/DREPTURI BĂNEȘTI ---
        {
            "tip_act": "PROCURĂ RIDICARE PENSIE",
            "scop": "ridicarea pensiei lunare și a taloanelor",
            "extra": {
                "institutie": "Casa Județeană de Pensii",
                "oficiu_postal": "Oficiul Poștal din raza de domiciliu",
                "valabilitate": "până la revocare dar nu mai mult de 3 ani"
            }
        },

        # --- SCENARIUL 9: FISCAL (ANAF) ---
        {
            "tip_act": "PROCURĂ REPREZENTARE FISCALĂ",
            "scop": "obținere certificat atestare fiscală și depunere declarații",
            "extra": {
                "institutie": "Direcția Taxe și Impozite Locale și ANAF",
                "motiv": "vânzare imobil și regularizare taxe"
            }
        },

        # --- SCENARIUL 10: ADMINISTRATIV (ACTE STARE CIVILĂ) ---
        {
            "tip_act": "PROCURĂ SPECIALĂ",
            "scop": "obținere duplicat certificat de naștere/căsătorie",
            "extra": {
                "act_cerut": random.choice(["Certificat de Naștere", "Certificat de Căsătorie", "Cazier Judiciar"]),
                "autoritate": "Serviciul de Stare Civilă / Poliția Română"
            }
        }
    ]

    # Alegem un scenariu aleatoriu
    scenariu = random.choice(scenarii)

    # Construim obiectul metadata final
    meta = {
        "tip_act": scenariu["tip_act"],
        "scop": scenariu["scop"],
        "mandant": {
            "nume": fake.name(),
            "cnp": fake.ssn(),
            "adresa": fake.address().replace("\n", ", ")
        },
        "mandatar": {
            "nume": fake.name(),
            "cnp": fake.ssn(),
            "adresa": fake.address().replace("\n", ", ")
        },
        "data_semnarii": datetime.now().strftime("%d.%m.%Y"),
        # Adăugăm datele extra direct în rădăcină pentru ca LLM-ul să le vadă ușor
        **scenariu["extra"]
    }

    return meta

# Generator

In [ ]:
def generator(meta):
    exemplu_text = get_template_procura(meta)

    prompt = f"""
SARCINĂ: Redactează textul final pentru o procură notarială.
Folosește structura și limbajul juridic din textul de mai jos, modificând detaliile minore dacă este necesar pentru coerență.

TEXT SURSĂ:
{exemplu_text}

INSTRUCȚIUNE: Returnează textul complet al procurii.
"""
    return run_llm([{"role": "user", "content": prompt}], max_new=900)

In [ ]:
import json
import re

def genereaza_scenariu_llm():
    """
    Folosește LLM-ul pentru a inventa un scenariu notarial complex și a-l returna ca JSON.
    """

    prompt = """
Ești un Generator de Date Sintetice pentru domeniul juridic.
SARCINA TA: Inventează un scenariu realist pentru un act notarial (procură sau declarație).
Fii creativ! Variază tipul actului (imobiliar, auto, succesiune, divorț, firmă, declarație călătorie).

TREBUIE SĂ RĂSPUNZI DOAR CU UN JSON VALID.
Respectă cu strictețe această structură (schema):

{
    "tip_act": "Tipul documentului (ex: PROCURĂ SPECIALĂ IMOBILIARĂ)",
    "scop": "Descrierea scurtă a mandatului",
    "mandant": {
        "nume": "Nume complet (inventat)",
        "cnp": "CNP valid (inventat)",
        "adresa": "Adresa completă (inventată)"
    },
    "mandatar": {
        "nume": "Nume complet (inventat)",
        "cnp": "CNP valid (inventat)",
        "adresa": "Adresa completă (inventată)"
    },
    "detalii_extra": {
        "descriere": "Orice detaliu specific necesar (ex: marca mașinii, adresa imobilului, numele minorului, numele băncii etc.)"
    }
}

REGULI:
1. Inventeaza scenarii plauzibile si logice.
2. Inventează date plauzibile (serii șasiu, numere cadastrale).
3. Nu scrie nimic înainte sau după JSON.
4. Nu te repeta.
"""

    # Folosim o temperatură mai mare (0.85) pentru creativitate și diversitate
    raw_res = run_llm([{"role": "user", "content": prompt}], max_new=600, temp=0.85)

    # Parser robust pentru JSON
    try:
        # Curățăm markdown-ul (```json ... ```)
        clean_res = raw_res.replace("```json", "").replace("```", "").strip()

        # Găsim acoladele
        start = clean_res.find("{")
        end = clean_res.rfind("}") + 1

        if start != -1:
            json_obj = json.loads(clean_res[start:end])

            # Asigurăm compatibilitatea cu scriptul de generare documente
            # (Mutăm detaliile extra în rădăcină, ca să fie ușor accesibile)
            if "detalii_extra" in json_obj:
                extra = json_obj.pop("detalii_extra")
                json_obj.update(extra)

            return json_obj

    except Exception as e:
        print(f" Eroare generare scenariu JSON: {e}")
        print(f"   Raw output: {raw_res[:100]}...")

    # Fallback: Dacă LLM-ul greșește JSON-ul, returnăm un scenariu default ca să nu crăpe
    return {
        "tip_act": "PROCURĂ GENERALĂ",
        "scop": "administrare bunuri",
        "mandant": {"nume": "Fallback User", "cnp": "1800101000000", "adresa": "București"},
        "mandatar": {"nume": "Fallback Agent", "cnp": "1800101000000", "adresa": "București"}
    }

In [ ]:
def run_llm(messages, max_new=512, temp=0.7):
    # Pregătire input
    input_ids = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    ).to(model.device)

    attention_mask = torch.ones_like(input_ids)

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new,
        do_sample=True,
        temperature=temp,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)



# Critic

In [ ]:
def critic(text, meta):
    """
    Verifică într-un singur pas atât structura (Forma) cât și conținutul (Scopul).
    """
    scop_cerut = meta.get('scop', 'Nespecificat')
    mandant_cerut = meta['mandant']['nume']

    prompt = f"""
Ești un AUDITOR NOTARIAL EXPERT.
Analizează documentul de mai jos și completează raportul de validare JSON.

DATE DE REFERINȚĂ (Ce trebuie să conțină):
1. Scopul Mandatului: "{scop_cerut}"
2. Mandant: "{mandant_cerut}"

TEXT DE VERIFICAT:
---
{text[:2000]}
--- (text trunchiat pentru analiză)

SARCINĂ DE AUDIT:
A. VERIFICARE FORMĂ:
   - Textul pare complet (are titlu, final)?
   - S-au eliminat toate placeholder-ele gen "...", "___", "[Data]"? (CRITIC!)
   - Există numele mandantului?

B. VERIFICARE SCOP:
   - Textul menționează clar acțiunea cerută ("{scop_cerut}")? (Atenție la sinonime juridice).

RĂSPUNDE DOAR CU ACEST JSON STRICT:
{{
    "forma_ok": (bool - false dacă există "..." sau lipsesc părți),
    "scop_ok": (bool - true dacă scopul e acoperit),
    "scor_calitate": (int 1-10),
    "observatii": "scurtă explicație a erorilor sau OK"
}}
"""
    # Temperatură mică pentru rigoare
    raw_res = run_llm([{"role": "user", "content": prompt}], max_new=300, temp=0.2)

    print(f"   [RAȚIONAMENT CRITIC]: {raw_res}") # Debug

    try:
        # Curățare JSON (metoda robustă)
        clean_res = raw_res.replace("```json", "").replace("```", "").strip()
        start = clean_res.find("{")
        end = clean_res.rfind("}") + 1

        if start != -1:
            return json.loads(clean_res[start:end])
        return None
    except:
        return None

# Cleaning up the data

In [ ]:
import os
import glob
from google.colab import drive

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Google Drive este deja conectat.")

# Folderul sursă specificat de tine
folder_input = "/content/drive/MyDrive/Colab Notebooks/procura"

# Folderul unde salvăm datele curate (creăm un subfolder ca să fie ordine)
folder_output = "/content/drive/MyDrive/Colab Notebooks/procura_cleaned"

# Creăm folderul de output dacă nu există
if not os.path.exists(folder_output):
    os.makedirs(folder_output)
    print(f"Am creat folderul pentru rezultate: {folder_output}")


def curata_document_cu_llm(text_murdar):
    """
    Trimite textul cu '...' la LLM și îi cere să îl completeze coerent.
    """
    prompt = f"""
SARCINĂ: Ești un asistent juridic expert.
Ai primit un document ciornă care conține spații libere marcate cu "..." sau puncte.
Rescrie textul completând TOATE spațiile libere cu DATE FICTIVE, dar plauzibile pentru context (nume, date, CNP-uri, adrese).

REGULI:
1. Elimină absolut toate "..." și înlocuiește-le cu date concrete.
2. Nu folosi paranteze pătrate [Nume] în textul final, ci pune direct un nume (ex: Popescu Ion).
3. Păstrează structura juridică intactă.
4. Returnează DOAR textul curățat.

TEXT ORIGINAL:
{text_murdar}
"""
    # Apelăm modelul (presupunem că run_llm e deja definită în celulele anterioare)
    return run_llm([{"role": "user", "content": prompt}], max_new=1024, temp=0.4)

fisiere_gasite = glob.glob(f"{folder_input}/*.txt")

print(f"\nCaut în: {folder_input}")
print(f"Am găsit {len(fisiere_gasite)} fișiere text (.txt).")

if len(fisiere_gasite) == 0:
    print("ATENȚIE: Nu am găsit niciun fișier .txt în acest folder!")
    print("Asigură-te că modelele tale sunt salvate ca Text Document (.txt) în Drive.")
else:
    print("Încep procesarea...\n")

    for i, cale_fisier in enumerate(fisiere_gasite):
        nume_fisier = os.path.basename(cale_fisier)
        print(f"[{i+1}/{len(fisiere_gasite)}] Procesez: {nume_fisier}...")

        try:
            # Citire
            with open(cale_fisier, 'r', encoding='utf-8') as f:
                text_brut = f.read()

            # Curățare cu AI
            text_curat = curata_document_cu_llm(text_brut)

            # Salvare pe Drive
            cale_salvare = os.path.join(folder_output, f"CURAT_{nume_fisier}")
            with open(cale_salvare, 'w', encoding='utf-8') as f:
                f.write(text_curat)

            print(f"  Salvat: Dataset_Procuri_Curate/CURAT_{nume_fisier}")

        except Exception as e:
            print(f" Eroare: {e}")

    print(f"\n Gata! Toate documentele curățate sunt în: {folder_output}")

# Preparing the data for selection

In [ ]:
import os
import glob
import random
from google.colab import drive

# 1. Conectare Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Încărcare modele în memorie
folder_curate = "/content/drive/MyDrive/Colab Notebooks/procura_cleaned"
sabloane_juridice = []

print(f"Se încarcă șabloanele din: {folder_curate}")
fisiere = glob.glob(f"{folder_curate}/*.txt")

if not fisiere:
    print("EROARE: Nu am găsit fișiere .txt! Asigură-te că ai rulat pasul de curățare.")
else:
    for cale in fisiere:
        with open(cale, 'r', encoding='utf-8') as f:
            text = f.read()
            # Salvăm textul și numele fișierului (ca să știm ce tip e)
            sabloane_juridice.append({
                "nume_fisier": os.path.basename(cale).lower(),
                "continut": text
            })
    print(f"Am memorat {len(sabloane_juridice)} modele juridice gata de folosire.")

In [ ]:
def alege_sablon_potrivit(meta,scop_document):
    """
    Caută în lista de șabloane unul care se potrivește cu scopul cerut.
    """
    cuvinte_cheie = scop_document.lower().split()

    # 1. Căutare exactă
    for sablon in sabloane_juridice:
        # Verificăm dacă vreun cuvânt cheie apare în numele fișierului sau în conținut
        matches = sum(1 for word in cuvinte_cheie if word in sablon["continut"].lower())
        if matches > 0:
            return sablon["continut"]

    # 2. Fallback: Dacă nu găsim nimic specific, returnăm unul aleatoriu
    return get_template_procura(meta)

# Procura generation

In [ ]:
def genereaza_document_strict(meta):
    # 1. Alegem șablonul (Contextul)
    text_sablon = alege_sablon_potrivit(meta,meta['tip_act'] + " " + meta.get('scop', ''))

    # 2. Construim Prompt-ul "One-Shot"
    prompt = f"""
SARCINĂ: Ești un notar public.
Trebuie să redactezi un document nou, folosind structura și limbajul juridic din ȘABLONUL de mai jos.
Nu schimba formulările juridice si nu te repeta.

--- ȘABLON DE REFERINȚĂ (Inspira-te dupa structura de aici) ---
{text_sablon[:2500]}
--- SFÂRȘIT ȘABLON ---

DATE NOI DE FOLOSIT PENTRU NOUL DOCUMENT:
- Tip act: {meta['tip_act']}
- Părți: {meta['mandant']['nume']} (CNP {meta['mandant']['cnp']}) -> {meta['mandatar']['nume']}
- Detalii specifice: {json.dumps(meta, ensure_ascii=False)}

REZULTAT FINAL (Doar textul documentului):
"""
    return run_llm([{"role": "user", "content": prompt}], max_new=1024)

In [ ]:
import torch
import random
import json
import re
from datetime import datetime, timedelta


print("\n Începem generarea a 5 documente de test...\n")
corpus = []

for i in range(5):
    print(f"--- Document {i+1} ---")
    meta = genereaza_scenariu_llm()


    print("Se scrie procura...")
    text = genereaza_document_strict(meta)

    print("Ce s-a generat:\n")
    print(text)

    print("Se verifică (Critic)...")
    evaluare = critic(text, meta)

    if evaluare:
        corpus.append({"meta": meta, "text": text, "eval": evaluare})
    else:
        print("Eroare parsare critic (Verifică [RAW CRITIC] mai sus).")

print(f"\n Gata! {len(corpus)} documente procesate.")

In [ ]:
print(corpus[3])

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/corpus_full_synthetic.jsonl", "a", encoding="utf-8") as f:
    for doc in corpus:
        f.write(json.dumps(doc, ensure_ascii=False) + "\n")